# Self-ask with search

This walkthrough showcases the self-ask with search chain.

In [1]:
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

## Using LangChain Expression Language

First we will show how to construct this agent from components using LangChain Expression Language

In [2]:
from langchain.agents.output_parsers import SelfAskOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain import hub

In [16]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [12]:
llm_with_stop = llm.bind(stop=["\nIntermediate answer:"])

In [13]:
agent = {
    "input": lambda x: x["input"],
    # Use some custom observation_prefix/llm_prefix for formatting
    "agent_scratchpad": lambda x: format_log_to_str(
        x['intermediate_steps'], 
        observation_prefix="\nIntermediate answer: ",
        llm_prefix="",
    ),
} | prompt | llm_with_stop | SelfAskOutputParser()

In [ ]:
from langchain.agents import AgentExecutor

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [15]:
agent_executor.invoke({"input": "What is the hometown of the reigning men's U.S. Open champion?"})



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the reigning men's U.S. Open champion?Men's US Open Tennis Champions Novak Djokovic earned his 24th major singles title against 2021 US Open champion Daniil Medvedev, 6-3, 7-6 (7-5), 6-3. The victory ties the Serbian player with the legendary Margaret Court for the most Grand Slam wins across both men's and women's singles.
Follow up: Where is Novak Djokovic from?Belgrade, Serbia
So the final answer is: Belgrade, Serbia

> Finished chain.


{'input': "What is the hometown of the reigning men's U.S. Open champion?",
 'output': 'Belgrade, Serbia'}

## Use off-the-shelf agent

In [8]:
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
self_ask_with_search.run(
    "What is the hometown of the reigning men's U.S. Open champion?"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: Who is the reigning men's U.S. Open champion?
Intermediate answer: Men's US Open Tennis Champions Novak Djokovic earned his 24th major singles title against 2021 US Open champion Daniil Medvedev, 6-3, 7-6 (7-5), 6-3. The victory ties the Serbian player with the legendary Margaret Court for the most Grand Slam wins across both men's and women's singles.

Follow up: Where is Novak Djokovic from?
Intermediate answer: Belgrade, Serbia
So the final answer is: Belgrade, Serbia

> Finished chain.


'Belgrade, Serbia'